In [1]:
import pandas as pd
import numpy as np

import datetime as dt
import string

import matplotlib.pyplot as plt

In [2]:
INFILE = '../build_data/spx_data_20240614.xlsx'

In [3]:
mkt_raw = pd.read_excel(INFILE,sheet_name='benchmarks prices').set_index('date')
spx_raw = pd.read_excel(INFILE,sheet_name='spx prices adjusted').set_index('date')

In [4]:
info_spx = pd.read_excel(INFILE,sheet_name='spx names').set_index('ticker')
info_mkt = pd.read_excel(INFILE,sheet_name='market names').set_index('ticker')

In [5]:
DT0 = '2016'
AGG = 'W-FRI'
FREQ = 52

def filter_prices(prices,DT0,AGG):
    pxT = prices.loc[DT0:,].copy()
    resamp = pxT.resample(AGG).last().iloc[:-1,:]
    rets = resamp.pct_change().dropna(axis=0)    
    return rets

rets_spx = filter_prices(spx_raw.dropna(axis=1),DT0,AGG)
rets_mkt = filter_prices(mkt_raw,DT0,AGG)
rets_spx, rets_mkt = rets_spx.align(rets_mkt, join='inner',axis=0)

rets_mkt.rename(columns={'XBTUSD BGN Curncy':'BTC'},inplace=True)

In [6]:
OUTFILE = '../data/spx_returns_cleaned.xlsx'

with pd.ExcelWriter(OUTFILE) as writer:  
    info_spx.to_excel(writer, sheet_name= 's&p500 names')
    info_mkt.to_excel(writer, sheet_name= 'benchmark names')
    
    rets_spx.to_excel(writer, sheet_name=f's&p500 rets')
    rets_mkt.to_excel(writer, sheet_name='benchmark rets')